# Programming Task Description

## Programming Task: Implementing a Character-Level GPT Model

### Introduction
In this task, you will create a Python script using PyTorch to implement a simplified GPT (Generative Pre-trained Transformer) model for character-level language modeling. The model will be trained on the text in input.txt to predict the next character in a sequence and generate new text based on a given context. The architecture follows the decoder part of the transformer model from the "Attention is All You Need" paper by Vaswani et al., focusing on masked multi-head self-attention to ensure predictions depend only on previous positions.

## Task Description
### Your goal is to write a Python jupyter notebook that:

1. Reads and processes the text from input.txt.
2. Implements a decoder-only transformer model with manual attention mechanisms.
3. Trains the model on the processed data.
4. Generates new text using the trained model.

You will use PyTorch and implement the attention mechanism from scratch, following the decoder structure outlined in the "Attention is All You Need" paper.

### Step-by-step Guide

1. Data Preparation
* Read all text from input.txt using UTF-8 encoding.
* Create a sorted list of unique characters (vocabulary) from the text.
* Build two dictionaries:
    * stoi: Maps characters to integers (e.g., 'a' -> 0).
    * itos: Maps integers to characters (e.g., 0 -> 'a').
* Define functions:
    * encode(s): Converts a string to a list of integers using stoi.
    * decode(l): Converts a list of integers to a string using itos.
* Encode the entire text into a tensor of integers using torch.tensor.
* Split the data: 90% for training, 10% for validation.

2. Data Loading
* Implement a function get_batch(split):
    * Input: split is either 'train' or 'val'.
    * Select the appropriate dataset (training or validation).
    * Randomly sample batch_size starting indices, ensuring each sequence fits within block_size.
* Return:
    * x: A tensor of shape (batch_size, block_size) with input sequences.
    * y: A tensor of shape (batch_size, block_size) with target sequences (shifted by one position).
* Move tensors to the device (CPU or GPU).

3. Model Architecture
* Implement the following components in a decoder-only transformer:
    * Embedding Layers:
        * Token embedding: nn.Embedding(vocab_size, n_embd) for character indices.
        * Position embedding: nn.Embedding(block_size, n_embd) for positions 0 to block_size-1.
    * Transformer Blocks:
        * Each block includes:
            * Masked Multi-Head Self-Attention:
                * Implement manually (do not use nn.MultiheadAttention).
                * For each head:
                    * Linear layers for queries (Q), keys (K), and values (V).
                    * Scaled dot-product attention: attention = softmax((Q @ K.T) / sqrt(d_k)) @ V.
                    * Mask future positions with a lower triangular matrix (e.g., tril) by setting future weights to -inf before softmax.
                * Concatenate heads and apply a projection layer.
            * Feed-Forward Network: nn.Linear(n_embd, 4 * n_embd) → ReLU → nn.Linear(4 * n_embd, n_embd).
            * Layer Normalization: Apply nn.LayerNorm(n_embd) before each sub-layer (pre-norm).
            * Residual Connections: Add input to output of each sub-layer.
        * Use n_layer blocks in sequence.
    * Final Layers:
        * nn.LayerNorm(n_embd) for final normalization.
        * nn.Linear(n_embd, vocab_size) to produce logits.
* Define a GPTLanguageModel class with:
    * forward(idx, targets=None): Computes logits and loss (if targets provided).
    * generate(idx, max_new_tokens): Autoregressively generates new tokens.

4. Training
* Use the AdamW optimizer with learning_rate = 3e-4.
* Train for max_iters = 5000 iterations.
* Estimate and print training and validation losses:
* Compute loss using F.cross_entropy on flattened logits and targets.

5. Text Generation
* Implement generate(idx, max_new_tokens):
    * Start with an initial context idx (shape (B, T)).
    * For max_new_tokens steps:
        * Crop idx to the last block_size tokens.
        * Get logits from forward.
        * Apply softmax to the last time step’s logits to get probabilities.
        * Sample the next token using torch.multinomial.
        * Append the sampled token to idx.
    * Return the extended sequence.

### Hyperparameters
Use these values:

* batch_size = 64
* block_size = 256
* n_embd = 384
* n_head = 6
* n_layer = 6
* dropout = 0.2
* learning_rate = 3e-4
* max_iters = 5000

### Understanding the Decoder
The "Attention is All You Need" paper describes a transformer with an encoder and decoder. For this task, you focus on the decoder-only architecture used in GPT:

* Masked Self-Attention: Ensures the model only attends to previous positions in the sequence, making it autoregressive. This is achieved by masking future tokens in the attention computation, as shown below:

$Attention (Q, K, V) = softmax((Q@K.T)/sqrt(d_{k}) + mask) @V$

where $mask$ sets future positions to $-inf$

* Decoder Role: In the original paper, the decoder generates output sequences while attending to the encoder’s output. Here, without an encoder, it uses self-attention on the input sequence alone, predicting the next token step-by-step.

### Additional Notes
* Manual Attention: Implement attention from scratch to understand its mechanics (no pre-built PyTorch modules).
* Masking: Use a lower triangular matrix (e.g., torch.tril) to mask future positions.
* Device Handling: Set device = 'cuda' if torch.cuda.is_available() else 'cpu' and move tensors/models accordingly.
* Dropout: Apply nn.Dropout(dropout) in attention and feed-forward layers for regularization.

### Deliverables
A Python script that:
* Implements all steps above.
* Prints training and validation losses every 500/100? iterations (up to you).
* Generates and prints a 500-character sample after training.

### Evaluation Criteria
* Correct data preparation and batch loading.
* Accurate implementation of the transformer model, especially masked self-attention.
* Successful training with decreasing loss.
* Generation of coherent (for character-level) text.

# Installing Packages
---

In [1]:
!pip install torch

# Data Preprocessing
---

In [2]:
import torch

def read_file_utf8(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# reading text file with utf8 encoding
file_path = 'input.txt'
text = read_file_utf8(file_path)

# sorted list of unique characters from input
vocabulary = sorted(list(set(text)))

# string to integer mapping`
stoi = {ch: i for i, ch in enumerate(vocabulary)}

# integer to string mapping
itos = {i: ch for i, ch in enumerate(vocabulary)}

**Encode Decode Functions**

In [3]:
def encode(s):
  output = []
  for char in s:
    output.append(stoi[char])
  return output

def decode(i):
  output = ""
  for j in i:
    output += itos[j]
  return output

**Tokenization and Training-Val Split (90/10)**

In [4]:
from collections import Counter

# character-level tokenization
tokens = encode(text)
tensor = torch.tensor(tokens) # 1D tensor of size [1115394] elements
print(f"Tensor size: {tensor.shape}")

split_idx = int(0.9 * len(tensor))

# split tensor into 90/10 training and validation sets
train_data = tensor[:split_idx]
print(f"Training set size: {train_data.shape}")
val_data = tensor[split_idx:]
print(f"Validation set size: {val_data.shape}")

Tensor size: torch.Size([1115394])
Training set size: torch.Size([1003854])
Validation set size: torch.Size([111540])


# Data Loading
---

**Get Batch Function**

In [5]:
batch_size = 64 # num of blocks
block_size = 256 # num of chars in a sequence

def get_batch(split):

  # returns random int tensor of size batch_size
  rand_indices = torch.randint(len(split) - block_size, (batch_size,))

  # returns tensors of size (batch_size, block_size)
  x = torch.stack([split[i:i+block_size] for i in rand_indices])

  # target sequence shifted by 1
  y = torch.stack([split[i+1:i+block_size+1] for i in rand_indices])
  return x, y

# Model Architecture
---

**Embedding Layer**

In [6]:
from torch import nn
from torch.nn import functional as F
from math import sqrt

class TransformerEmbedding(nn.Module):
    def __init__(self, vocab_size, n_embd, block_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.position_embedding = nn.Embedding(block_size, n_embd)

    def forward(self, x):
        B, T = x.shape  # x is (batch, seq_len) of token indices
        tok_emb = self.token_embedding(x)         # (B, T, n_embd)
        pos_emb = self.position_embedding(torch.arange(T, device=x.device))  # (T, n_embd)
        return tok_emb + pos_emb  # (B, T, n_embd)

**Transformer Block**

In [7]:
class Transformer_Block(nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, n_layer, block_size, dropout):
        super().__init__()

        # n_heads hyperparameter
        self.n_head = n_head

        # embedding layer
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # (num of embeddings, dimensions)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        # linear layers for multi_head attention
        self.q_proj = nn.Linear(n_embd, n_embd)
        self.k_proj = nn.Linear(n_embd, n_embd)
        self.v_proj = nn.Linear(n_embd, n_embd)
        self.output_proj = nn.Linear(n_embd, n_embd)

        # feed forward layer
        self.feed_forward = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd)
        )

        # norm layers
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

        # dropout
        self.dropout = nn.Dropout(dropout)


    def masked_attention(self, Q, K, V):
        """
        Q, K, V: (batch, n_head, seq_len, head_dim)
        Returns: (batch, n_head, seq_len, head_dim)
        """

        # compute scores
        d_k = Q.shape[-1]
        scores = (Q @ K.transpose(-2, -1)) / sqrt(d_k)

        # creates a diagonal of -inf values to mask future comparisions
        T = scores.size(-1)
        mask = torch.triu(torch.ones(T, T), diagonal=1).to(Q.device).bool()
        scores = scores.masked_fill(mask, float('-inf'))

        # apply softmax
        weights = F.softmax(scores, dim=-1)
        return weights @ V # returns tensor

    def multihead_attention(self, x):
        """
        x: (batch, tokens, embeddings)
        Returns: (batch, tokens, embeddings)
        """
        # batch(num of sequences), tokens(num of tokens), embeddings(size of vector for each token)
        B, T, E = x.shape

        # divide embeddings for parallel attention
        head_dim = E // self.n_head

        # passing Q, K, V through linear layers to transform their shapes
        Q = self.q_proj(x) # (B, T, E)
        K = self.k_proj(x)
        V = self.v_proj(x)

        # using view to add n_head as a dimension
        Q = Q.view(B, T, self.n_head, head_dim).transpose(1, 2)
        K = K.view(B, T, self.n_head, head_dim).transpose(1, 2)
        V = V.view(B, T, self.n_head, head_dim).transpose(1, 2)

        output = self.masked_attention(Q, K, V)                     # (B, n_head, T, head_dim)
        output = output.transpose(1, 2).contiguous().view(B, T, E)  # (B, T, E)

        # go through final linear layer
        output = self.output_proj(output)

        # dropout for regularlization
        output = self.dropout(output)
        return output


    def forward(self, x):
        # masked multihead attention -> add & norm
        attn_out = self.multihead_attention(x)
        x = self.ln1(x + attn_out)

        # feedforward -> dropout -> add & norm
        ff_out = self.feed_forward(x)
        ff_out = self.dropout(ff_out)
        x = self.ln2(x + ff_out)
        return x

**GPTLanguageModel:**

In [8]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, n_layer, block_size, dropout):
        super().__init__()

        self.block_size = block_size
        self.vocab_size = vocab_size
        self.n_layer = n_layer
        self.n_head = n_head
        self.n_embd = n_embd
        self.dropout = dropout

        # embedding layer
        self.embedding = TransformerEmbedding(vocab_size, n_embd, block_size)

        # transformer blocks
        self.blocks = nn.Sequential(*[
            Transformer_Block(
                vocab_size=vocab_size,         
                n_embd=n_embd,
                n_head=n_head,
                n_layer=n_layer,               
                block_size=block_size,
                dropout=dropout
            )
            for _ in range(n_layer)
        ])

        # norm and feedforward layer 
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        """
        idx: (B, T) token indices
        targets: (B, T) optional ground truth for loss
        """
        x = self.embedding(idx)  # (B, T, n_embd)
        x = self.blocks(x)       # (B, T, n_embd)
        x = self.ln_f(x)         # (B, T, n_embd)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            return logits, None
        
        # reshape logits and targets for cross entropy
        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)
        return logits, loss    


    def generate(self, idx, max_new_tokens):
        """
        idx: (B, T) starting tokens
        Returns: (B, T + max_new_tokens) — generated sequence
        """
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]  # crop to block size
            logits, _ = self.forward(idx_cond)    # (B, T, vocab_size)
            next_token_logits = logits[:, -1, :]  # last time step
            probs = F.softmax(next_token_logits, dim=-1)  # (B, vocab_size)
            next_token = torch.multinomial(probs, num_samples=1)  # (B, 1)
            idx = torch.cat((idx, next_token), dim=1)  # append to sequence
        return idx


**Initialize Model with Hyperparameters**

In [9]:
# Hyperparameters
# * batch_size = 64
# * block_size = 256
# * n_embd = 384
# * n_head = 6
# * n_layer = 6
# * dropout = 0.2
# * learning_rate = 3e-4
# * max_iters = 5000

model = GPTLanguageModel(
    vocab_size=len(vocabulary),
    n_embd=384,
    n_head=6,
    n_layer=6,
    block_size=256,
    dropout=0.2
)

**Setting Tensor to Device (GPU or CPU)**


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

# setting training and validation data to use device
train_data = tensor.to(device) 
val_data = tensor.to(device)
model = model.to(device)

print(train_data.device, val_data.device, next(model.parameters()).device)

Using cuda
cuda:0 cuda:0 cuda:0


# Training
---

**Initialize esimate loss function for train and val sets**

In [11]:
@torch.no_grad()
def estimate_loss():
    model.eval()
    losses = {'train': 0.0, 'val': 0.0}
    for split in ['train', 'val']:
        data = train_data if split == 'train' else val_data
        for _ in range(5): # run 5 testing batches
            x, y = get_batch(data)
            _, loss = model(x, y)
            losses[split] += loss.item()
        losses[split] /= 5 # get the average out of 5 runs
    model.train()
    return losses

In [12]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

max_iters = 5000

for step in range(max_iters):
    xb, yb = get_batch(train_data)
    logits, loss = model(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    
    if step % 100 == 0:
        losses = estimate_loss()
        print(f"Step: {step}, Train Loss: {losses['train']:.4f}, Val Loss: {losses['val']:.4f}")



Step: 0, Train Loss: 3.7468, Val Loss: 3.7425
Step: 100, Train Loss: 2.4762, Val Loss: 2.4781
Step: 200, Train Loss: 2.3762, Val Loss: 2.3878
Step: 300, Train Loss: 2.1555, Val Loss: 2.1554
Step: 400, Train Loss: 1.9553, Val Loss: 1.9562
Step: 500, Train Loss: 1.8002, Val Loss: 1.7954
Step: 600, Train Loss: 1.7049, Val Loss: 1.7052
Step: 700, Train Loss: 1.6178, Val Loss: 1.6275
Step: 800, Train Loss: 1.5563, Val Loss: 1.5483
Step: 900, Train Loss: 1.4961, Val Loss: 1.4938
Step: 1000, Train Loss: 1.4643, Val Loss: 1.4649
Step: 1100, Train Loss: 1.4063, Val Loss: 1.4068
Step: 1200, Train Loss: 1.4007, Val Loss: 1.4000
Step: 1300, Train Loss: 1.3625, Val Loss: 1.3701
Step: 1400, Train Loss: 1.3434, Val Loss: 1.3416
Step: 1500, Train Loss: 1.3121, Val Loss: 1.3171
Step: 1600, Train Loss: 1.3058, Val Loss: 1.3101
Step: 1700, Train Loss: 1.2962, Val Loss: 1.3065
Step: 1800, Train Loss: 1.2797, Val Loss: 1.2675
Step: 1900, Train Loss: 1.2594, Val Loss: 1.2582
Step: 2000, Train Loss: 1.2396, 

# Model Testing
--- 

In [14]:
# generate sample output
model.eval()
context = torch.zeros((1, 1), dtype=torch.long).to(device)  # start token (e.g. 0)
generated = model.generate(context, max_new_tokens=500)[0].tolist()
print("Sample output:", decode(generated))

Sample output: 
How came this shame man thus ice? why my as
I could not, first a brace of king, thou
shalt shay the fender of heart's child? yes, but I
know to have this visitation.

LUCIO:
Know, how you will it be hooked?

ISABELLA:
Hope is foolish desire
Your will be perceive in the virtuous proclaimedy?

DUKE VINCENTIO:
Ay, why, there's a gage: stay a gaol.
Inch will death not, thou wilt lay with thee.

iPSON:
O, if he be it so, she had been never been.

DUKE OF AUMERLE:
He knew my soul! When she hath thy fa
